In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error

# Generate some sample data
data = pd.DataFrame({
    'Feature1': np.random.rand(100),
    'Feature2': np.random.rand(100),
    'Feature3': np.random.rand(100),
    'Feature4': np.random.rand(100),
    'Target': np.random.rand(100)
})

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train, test = data.iloc[:train_size], data.iloc[train_size:]

# ARIMA model
def arima_forecast(train, test):
    history = [x for x in train['Target']]
    predictions = []
    for t in range(len(test)):
        model = ARIMA(history, order=(5,1,0)) # ARIMA(5,1,0) model
        model_fit = model.fit()
        output = model_fit.forecast()
        yhat = output[0]
        predictions.append(yhat)
        obs = test['Target'].iloc[t]
        history.append(obs)
    return predictions

# ANN model
def ann_model(train, test):
    # Normalize data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_train = scaler.fit_transform(train)
    scaled_test = scaler.transform(test)

    # Prepare data
    X_train, y_train = scaled_train[:, :-1], scaled_train[:, -1]
    X_test, y_test = scaled_test[:, :-1], scaled_test[:, -1]

    # Define model
    model = Sequential()
    model.add(Dense(10, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Fit the model
    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

    # Make predictions
    predictions = model.predict(X_test)

    # Invert predictions
    predictions = scaler.inverse_transform(np.concatenate((X_test, predictions), axis=1))[:, -1]

    return predictions

# ARIMA-ANN Hybrid
def arima_ann_hybrid(train, test):
    # Get ARIMA predictions
    arima_preds = arima_forecast(train, test)
    
    # Combine ARIMA predictions with other features
    train_features = train.drop(columns=['Target'])
    test_features = test.drop(columns=['Target'])
    combined_train = pd.concat([train_features.reset_index(drop=True), pd.Series(arima_preds, name='ARIMA_Prediction')], axis=1)
    combined_test = pd.concat([test_features.reset_index(drop=True), pd.Series(arima_preds[-len(test):], name='ARIMA_Prediction')], axis=1)

    # ANN predictions
    ann_preds = ann_model(combined_train, combined_test)

    return ann_preds

# Make predictions using ARIMA-ANN Hybrid
predictions = arima_ann_hybrid(train, test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(test['Target'], predictions))
print(f'Root Mean Squared Error (RMSE): {rmse}')


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-03-22 23:38:33.819741: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


ValueError: Input contains NaN.